In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL.ImageColor import colormap
from fastai.tabular.all import *
from openai import OpenAI
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
data_all = pd.read_csv("/Users/holee/Desktop/GlobalYouTubeStatistics.csv", encoding="latin1")
data_all = data_all.drop(["Title","Youtuber","highest_yearly_earnings","highest_monthly_earnings","lowest_monthly_earnings"],axis=1)


In [3]:
data_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1006 entries, 0 to 1005
Data columns (total 24 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   rank                                     1006 non-null   int64  
 1   subscribers                              1003 non-null   float64
 2   video views                              1006 non-null   float64
 3   category                                 951 non-null    object 
 4   uploads                                  1006 non-null   int64  
 5   Country of origin                        881 non-null    object 
 6   Country                                  881 non-null    object 
 7   Abbreviation                             881 non-null    object 
 8   channel_type                             974 non-null    object 
 9   video_views_rank                         1005 non-null   float64
 10  country_rank                             887 non

In [4]:
data_all.isna().sum()

rank                                         0
subscribers                                  3
video views                                  0
category                                    55
uploads                                      0
Country of origin                          125
Country                                    125
Abbreviation                               125
channel_type                                32
video_views_rank                             1
country_rank                               119
channel_type_rank                           35
video_views_for_the_last_30_days            57
lowest_yearly_earnings                       0
subscribers_for_last_30_days               340
created_year                                 5
created_month                               12
created_date                                 5
Gross tertiary education enrollment (%)    126
Population                                 126
Unemployment rate                          126
Urban_populat

In [6]:
name_have_na = data_all.columns[data_all.isna().sum() > 0].tolist()
name_have_na

['subscribers',
 'category',
 'Country of origin',
 'Country',
 'Abbreviation',
 'channel_type',
 'video_views_rank',
 'country_rank',
 'channel_type_rank',
 'video_views_for_the_last_30_days',
 'subscribers_for_last_30_days',
 'created_year',
 'created_month',
 'created_date',
 'Gross tertiary education enrollment (%)',
 'Population',
 'Unemployment rate',
 'Urban_population',
 'Latitude',
 'Longitude']

In [7]:
for cc in name_have_na:
    if data_all[cc].dtype != "int64" or data_all[cc].dtype != "float64":
        null_index = np.where(pd.isnull(data_all[cc]) == True)[0]
        for i in null_index:
            replease_idx = np.abs(data_all['lowest_yearly_earnings'] - data_all.loc[i,'lowest_yearly_earnings']).drop(i).nsmallest(10).index.tolist()
            data_all.loc[i,cc] = data_all.loc[replease_idx,cc].mode()[0]
    else:
        for i in continue_idex:
            continue_idex = np.where(pd.isna(data_all[cc]) == True)[0]
            nerst_idex = np.abs(data_all.loc[i,"lowest_yearly_earnings"]- data_all["lowest_yearly_earnings"]).nsmallest(10).index.tolist()
            data_all.loc[i,cc] = data_all.loc[nerst_idex,cc].median()




In [8]:
for i in data_all.columns.tolist():
    if data_all[i].dtype == "O":
       data_all[i]  = data_all[i].astype("category")

In [9]:
countary_list = data_all["Country"].unique().tolist()

In [ ]:
len(countary_list

['india',
 'United States',
 'India',
 'Brazil',
 'Japan',
 'Russia',
 'South Korea',
 'United Kingdom',
 'Canada',
 'Argentina',
 'Andorra',
 'Chile',
 'Cuba',
 'El Salvador',
 'Pakistan',
 'Philippines',
 'Thailand',
 'Spain',
 'Colombia',
 'Barbados',
 'Mexico',
 'United Arab Emirates',
 'Saudi Arabia',
 'Indonesia',
 'Turkey',
 'Venezuela',
 'Kuwait',
 'Jordan',
 'Netherlands',
 'Singapore',
 'Australia',
 'Italy',
 'Germany',
 'France',
 'Sweden',
 'Afghanistan',
 'Ukraine',
 'Latvia',
 'Switzerland',
 'Vietnam',
 'Malaysia',
 'China',
 'Iraq',
 'Egypt',
 'Ecuador',
 'Morocco',
 'Peru',
 'Bangladesh',
 'Finland',
 'Samoa']

In [19]:
# client = OpenAI(api_key ="********"
# )
# prrpor = str(countary_list) + "按照人口数量对他排序，给出python的排序list的代码"
# response = client.responses.create(
# model="gpt-4.1-nano",
# input = prrpor
# )
# resultss = response.output_text
countries = [
    'China', 'India', 'United States', 'Indonesia', 'Pakistan', 'Brazil', 'Bangladesh',
    'Russia', 'Mexico', 'Japan', 'Philippines', 'Egypt', 'Vietnam', 'Turkey', 'Germany',
    'Thailand', 'United Kingdom', 'France', 'Italy', 'South Korea', 'Colombia', 'Spain',
    'Ukraine', 'Iraq', 'Argentina', 'Canada', 'Morocco', 'Saudi Arabia', 'Peru', 'Malaysia',
    'Afghanistan', 'Venezuela', 'Chile', 'Netherlands', 'Ecuador', 'Jordan', 'Cuba',
    'United Arab Emirates', 'Sweden', 'Australia', 'Switzerland', 'Kuwait', 'Singapore',
    'Finland', 'El Salvador', 'Barbados', 'Samoa', 'Latvia', 'Andorra'
]
new_countries  = []
for i in reversed(range(len(countries))):
    new_countries.append(countries[i])
new_countries
data_all["Country"] = data_all["Country"].cat.set_categories(new_countries,ordered=True)



### torch embedding
le = LabelEncoder()
data_encoded = le.fit_transform(data_all["Country"])
data_tensor = torch.LongTensor(data_encoded)  #
embedding = nn.Embedding(num_embeddings=len(countary_list), embedding_dim=2)
embedded = embedding(data_tensor)

embedded.detach().numpy()




array([[-1.1477815 , -1.7433997 ],
       [-0.26769724,  0.7622722 ],
       [-0.26769724,  0.7622722 ],
       ...,
       [-0.26769724,  0.7622722 ],
       [ 0.71364135,  0.10124112],
       [ 2.6034324 ,  0.7856478 ]], dtype=float32)

In [476]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [477]:
X = data_all.drop("lowest_yearly_earnings",axis=1)

In [478]:
y = np.log(data_all["lowest_yearly_earnings"] + 1)

In [479]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [483]:
model = LinearRegression()

In [484]:
model.fit(pd.get_dummies(X_train),y_train)

LinearRegression()

In [485]:
regression = pd.DataFrame(zip(data_all.columns,model.coef_))
rank = np.abs(regression.iloc[:,1]).sort_values(ascending=False).index.tolist()
regression.iloc[rank,:]

,0,1
17,created_date,4.338992e+00
13,lowest_yearly_earnings,1.326000e+00
23,Longitude,9.285606e-01
21,Urban_population,6.518236e-01
19,Population,5.904507e-01
18,Gross tertiary education enrollment (%),3.805494e-01
11,channel_type_rank,-3.752714e-01
20,Unemployment rate,2.640365e-01
22,Latitude,2.209298e-01
15,created_year,3.330094e-02


In [486]:
clf = xgb.XGBRegressor(enable_categorical=True)

In [487]:
clf.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [488]:
att = clf.get_booster()
feature_importence = att.get_score(importance_type="gain")


In [1]:
feature_importence_series = pd.Series(feature_importence)
feature_importence_sorted = feature_importence_series.sort_values(ascending=False)
feature_importence_sorted[feature_importence_sorted.values > 0.002]

NameError: name 'pd' is not defined